# Container Pre-marshalling Problem Enviroment
En el siguiente documento se documentará cómo es que funciona el CPMP Enviroment

## Valores Relevantes

state : Es la representación del estado actual, de tipo 'Yard' para permitir realizarle cambios al estado

showDebug : Valor booleano que dicta si mostrará información de Debug o no

max_step : Pasos máximos, cálculados por una solución greedy. La idea es que la red mejore lo que dicta el greedy en StackedYard.py

training :  booleano que dice si está entrenando o no. Más que nada para ver qué conjunto utilizará para trabajar.

fileStack :  Los archivos que contienen los estados

current_step : Paso actual del estado, la idea es que sea menor a max_step

last_reward : la última recompenza obtenida, utilizada para recompenzar la diferencia entre la actual recompenza y la anterior (last_reward)


## Acciones
La acción que entregará como output va a ser un número de 0 a x, dónde x será la anchura del estado

Este valor representa el stack en el cuál se debe realizar la acción. En la actualidad, la red identificará un stack y una función de greedy realizará el cambio en dicho stack seleccionado.

```py
self.action_space = spaces.Discrete(x) # de 0 a x
```

## Observaciones
La observación será un arreglo que contendrá:

1)  El estado actual del patio con los containers, primero los datos rellenados y luego los vacios (0).
Los valores equivalentes al container son numeros enteros pertenecientes a `[0,∞)`. 
La representación dentro de la observación los normalizara en un rango de `[0,1]`, utilizando una normalización min-max, dónde el minimo y el maximo dependera del  problema que se este resolviendo.

2) Además de los valores del estado en sí, se cuenta con un valor booleano (0 ó 1) que representará si una columna se encuentra en orden (1) o no (0).

3) Como ultimos datos a concatenar en la observación, tenemos dos valores que representarán el paso actual y los pasos máximos. Los pasos máximos dependerán de lo que diga el greedy al iniciar el problema, la representación dentro de la observación será `pasos_actuales/pasos_maximos` para calcular el paso en el que se encuentra y 1 (`pasos_maximos/pasos_maximos`) para calcular el valor máximo.
  
Así se vería la observación que alimentará a la red.
```
[[0.6        0.4        0.73333333 0.46666667 0.6        0.4
  0.13333333 1.         0.6        0.46666667 0.86666667 0.46666667
  0.13333333 0.26666667 0.4        1.         0.2        0.73333333
  0.2        0.8        0.53333333 1.         0.66666667 0.66666667
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         1.         1.         1.
  1.         0.         0.         0.         1.         1.
  0.         1.        ]]
```

## Demonstración
A continuación se explicarán funciones del ambiente con ejemplos, de modo que se pueda comprender mejor cómo es que opera.

Las funciones tendrán los mismos nombres de los métodos encontrados en `containeryard.py` y `yard.py`.

### Crear el estado y manipularlo
Para crear el estado, se utilizará `yard.py`, debido a que `containeryard.py` implementará dicha representación a un formato de gym para facilitar el uso en redes de aprendizaje por reforzamiento.

A continuación un ejemplo de cómo se crea el estado, con un ejemplo encontrado en este mismo directorio, `example_10_5_0_226.bay`.

### Representación del Patio de Containers (yard.py)
Este archivo será la capa que modificará directamente el estado, el ambiente de gym (`containeryard.py`) la utilizará para no trabajarlo directamente.

Los métodos que explicaremos son:

`isStackEmpty(i)`: Retornará si el stack i (de 0 a n stacks) se encuentra vacío o no (True/False). Este método será utilizado para saber si es posible sacar un container de dicho stack, ya que si se encuentra vacío no se podrá sacar nada.

`isStackFull(i)`: Retornará si el stack i (de 0 a n stacks) se encuentra lleno o no (True/False). Este método será utilizado para saber si es posible poner un container en dicho stack, ya que si se encuentra lleno, no se podrá poner nada sobre este.

`moveStack(src, dest)`: Este método utilizará los dos anteriores para ver si es posible realizar una acción. En caso de ser posible, realizará dicha acción y cambiará el estado.

`getAsObservation()`: Este método devolverá el estado en formato de observación (sin la normalización), es decir, entregará el estado con los containers NO vacíos primero, y luego pondrá los vacíos, por ejemplo quedaría así: `[4, 3, 6, 7, 4, 4, 0, 0, 0, 0, 0 , ....]`

`render(self)`: Este método mostrará el estado de una manera "humanamente-leíble"

`isSorted(i)`: Verá si un stack i está ordenado o no.


In [18]:
import numpy as np
import random as rand
import os
import math
import numpy as np

from containeryard.StackedYard import Layout, greedy_solve, read_file, select_destination_stack
from Constants import FILE_PATH

import gym
from gym import error, spaces, utils
from gym.utils import seeding

#Archivo de Prueba
TEST_FILE = r'./example_10_5_0_226.bay'

############################################################################################    
# From yard.py
class Yard():
    def __init__(self, file, fromFile = True):
        yardInfo = os.path.basename(file.name).split("_")
        self.x,self.y = int(yardInfo[1]), int(yardInfo[2])
        self.state = np.zeros(shape=(self.x,self.y), dtype=np.int)

        #Loads the data from the file.
        lines = file.readlines()
        for i in range(len(lines)):
            pos = 0
            #Preparing the line reading.
            for num in lines[i].replace("\n","").split(" "):
                if num.isdigit():
                    self.state[i][pos] = int(num)
                    pos = pos + 1

        #######END#####################

        self.max = np.amax(self.state)
        self.min = np.amin(self.state)

    def isStackEmpty(self, i):
        return self.state[i][0] == 0

    def isStackFull(self, i):
        return self.state[i][self.y-1] != 0

    def moveStack(self, src, dest):
        value = 0
        
        #---> Primero, verificamos que la accion se pueda realizar.
        if self.isStackFull(dest) or self.isStackEmpty(src):
            return False

        #---> Segundo, conseguimos y eliminamos el valor en top.
        for pos in range(self.y-1, -1, -1):
            if self.state[src][pos] > 0:
                value = self.state[src][pos]
                self.state[src][pos] = 0
                break

        #---> Dejamos el valor 
        for pos in range(self.y):
            if self.state[dest][pos] == 0:
                self.state[dest][pos] = value
                break
        return True

    def getAsObservation(self):
        stateCopy = np.array(self.state, copy=True, dtype=np.float)

        return np.concatenate( (stateCopy[np.nonzero(stateCopy)],stateCopy[np.nonzero(stateCopy == 0)]))

    def render(self):
        rend = np.rot90(self.state, k=1)
        print(rend)

    def isSorted(self, i):
        lastNum = 999999
        for num in np.array(self.state[i]):
            if num == 0:
                break
            if num > lastNum:
                return False
            lastNum = num
        return True

    def asLayout(self):
        layoutState = []
        for stack in self.state:
            s = stack[np.nonzero(stack)] # Get the non zero values
            if s.size <= 0:
                s = np.array([0])
            layoutState.append(s.tolist())

        return layoutState

    def isDone(self):
        for sort in self.getAllSorts():
            if not sort:
                return False
        return True

    def getAllSorts(self):
        sorted = np.zeros(self.x, dtype=np.bool)
        for i in range(self.x):
            sorted[i] = self.isSorted(i)
        return sorted


### Ambiente de CPMP (containeryard.py)
ContainerYard se utilizará para que la red pueda tomar la observación y realizar acciones en el estado. Más información sobre gym puede ser leída en su página web: https://gym.openai.com/

Los métodos que utilizaremos para explicar serán:
`reset()`: Reiniciará el problema actual, cargando otro (En el caso de este ejemplo, siempre cargará uno, pero en la realidad cargará el siguiente!!)

`step(action)`: Realizará una acción utilizando _take_action(action), después, calculará la recompensa del estado y entregará la observación, recompensa , un booleano definiendo si el estado es terminal y, opcionalmente, información de debug. 

`_take_action(action)`: Este método realizará la acción en el estado. Primero calculará lo que se demora el greedy en realizar la acción, y lo guardará para calcular la recompensa. Después, utilizará el método de `Yard`, `moveStack` para realizar la acción en el estado.

`_next_observation()`: Entregará la observación para la red, descrita anteriormente en el inicio de este archivo


In [19]:
############################################################################################    
# From containeryard.py
class ContainerYard(gym.Env):
    metadata = {'render.modes':['human']}

    state : Yard
    showDebug : bool
    max_stel : int
    training : bool
    fileStack : list
    current_step : int
    last_reward : int

    def __init__(self, showDebug = False, training=False):
        super(ContainerYard, self).__init__()

        ### START OF CONFIG ###
        self.showDebug = showDebug
        self.max_step = 10
        self.training = training

        #---> Creting the stack for files to use..
        self.fileStack = []
        
        ############################
        self.current_step = 0
        self.last_reward = 0

        # Start The Episode
        self.reset()

        #Action and Observation Space
        self.action_space = spaces.Discrete(self.state.x)
        self.observation_space = spaces.Box(low=-1, high=255, shape=(self.state.x*self.state.y + self.state.x + 2 ,), dtype=np.float_)

    def reset(self):
        #################################################################
        # ESTE METODO DE RESET SÓLO CARGARÁ EL ARCHIVO DE PRUEBAS
        # PERO EL ORIGINAL CARGARÁ TODOS LOS DISPONIBLES, VER containeryard.py
        # Esto es SOLO UN EJEMPLO
        #################################################################
        # Se carga el Yard desde el archivo designado en TEST_FILE
        self.state = Yard(open(TEST_FILE))

        # Aquí se cargará el mismo estado, pero en un layout de stacks. Esto sólo será utilizado para
        # Resolverlo usando greedy, de modo de sacar el máximo de pasos y la recompensa.
        self.layout = read_file(TEST_FILE, self.state.y)
        self.max_step = greedy_solve(self.layout)
        self.greedy_steps = self.max_step

        self.current_step = 0

        return self._next_observation()

    def step(self, action):
        #Taking Action!
        ret = self._take_action(action)

        #New Greedy Value.
        self.greedy_steps = greedy_solve(
            Layout(self.state.asLayout(), self.state.y)
        )

        self.current_step += 1


        formula_reward = np.exp(-(self.current_step + self.greedy_steps))

        reward = formula_reward - self.last_reward

        self.last_reward = formula_reward

        done = (self.state.isDone() or self.current_step >= self.max_step)

        if ret is False:
            #Could not make action, so we punish it.
            reward = -1

        obs = self._next_observation()

        return obs, reward, done, _

    def _take_action(self, action):

        layoutState = self.state.asLayout()
        dest = select_destination_stack(
            Layout(layoutState, self.state.y), 
            action
        )
        return self.state.moveStack(action, dest)

    def _next_observation(self):
        
        #Normalization and Generating the Yard Observation
        obs = self.state.getAsObservation()
        obsMax = self.state.max
        obsMin = self.state.min
        for i in range(len(obs)):
            obs[i] = (obs[i]-obsMin)/(obsMax-obsMin)

        
        #Misc Observation
        for i in range(self.state.x):
            obs = np.insert(obs, obs.size, 1 if self.state.isSorted(i) is True else 0)
        
        #Normalizated Values
        cStep = (self.current_step)/(self.max_step)
        mStep = (self.max_step)/(self.max_step)
        obs = np.insert(obs, obs.size, [cStep, mStep])
        
        #self.state.render()
        return obs

### Pruebas
Aquí realizaremos pruebas para que se comprenda el funcionamiento.

Iniciaremos el estado y probaremos sus funcionamientos básicos.

`gym_representation` será nuestro ambiente de gym, primero mostraremos con `.state.render()` cómo es que verá la persona el ambiente y con `._next_observation()` lo que ve la red.

In [39]:
gym_representation = ContainerYard()

print('PARA HUMANOS')
gym_representation.state.render()

print('\nPARA REDES')
print(gym_representation._next_observation())

PARA HUMANOS
[[ 2  0  0  3  0  0  0  0  0  3]
 [14  0  0  2  0  0  0  0  0 12]
 [ 8  0  0 14  0  0  0  0  0 14]
 [14  0  0  1  5  0  0  0  0  8]
 [ 8  0 13 13 15  0 15  0  0 13]]

PARA REDES
[0.53333333 0.93333333 0.53333333 0.93333333 0.13333333 0.86666667
 0.86666667 0.06666667 0.93333333 0.13333333 0.2        1.
 0.33333333 1.         0.86666667 0.53333333 0.93333333 0.8
 0.2        0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         1.         1.         0.
 1.         1.         1.         1.         1.         0.
 0.         1.        ]


Ahora, probaremos un par de funciones.
Comenzaremos probando realizar un movimiento sobre la columna numero 3 (Si contamos desde 0, sería la número 2.)

El método retornara cuatro valores:
La nueva observación, la recompensa, si el estado es terminal y, por último, información adicional opcional.

In [42]:
print("Información de Retorno:")
print(gym_representation.step(2))

print("\nRender:")
gym_representation.state.render()

Información de Retorno:
(array([0.53333333, 0.93333333, 0.53333333, 0.93333333, 0.13333333,
       0.86666667, 0.06666667, 0.93333333, 0.13333333, 0.2       ,
       1.        , 0.33333333, 1.        , 0.86666667, 0.86666667,
       0.53333333, 0.93333333, 0.8       , 0.2       , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 1.        , 1.        , 0.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 0.        ,
       0.2       , 1.        ]), -1, False, (array([0.53333333, 0.93333333, 0.53333333, 0.93333333, 0.13333333,
       0.86666667, 0.06666667, 0.93333333, 0.13333333, 0.2 

A continuación utilizaremos los métodos que se utilizan dentro de todas las funciones:

In [41]:
print("Stacks Ordenados: ")
print(gym_representation.state.getAllSorts())


print("\nEs Estado Terminal: ")
print(gym_representation.state.isDone())

print("\nValor Greedy Estado Actual - Pasos Tomados: ")
print(greedy_solve(Layout(gym_representation.state.asLayout(), gym_representation.state.y)), gym_representation.current_step)

print('\nEstado Actual:')
gym_representation.state.render()

Stacks Ordenados: 
[False  True  True False  True  True  True  True  True False]

Es Estado Terminal: 
False

Valor Greedy Estado Actual - Pasos Tomados: 
11 1

Estado Actual:
[[ 2  0  0  3  0  0  0  0  0  3]
 [14  0  0  2  0  0  0  0  0 12]
 [ 8  0  0 14  0  0  0  0  0 14]
 [14  0  0  1  5  0 13  0  0  8]
 [ 8  0  0 13 15  0 15  0  0 13]]
